# The runbook to detect the data type changes of dataset and remove the broken filter of the analysis

## Define the parameters

In [ ]:
account_id = <<aws account id>>
new_dataset_id = << data set id >>
region = 'us-east-1'

## Back up the dataset description 

In [66]:
import boto3
import json
import os
from datetime import datetime
def get_dataset_description_from_api(account_id, dataset_id, region='us-east-1'):
    client = boto3.client('quicksight', region_name=region)
    response = client.describe_data_set(
        AwsAccountId=account_id,
        DataSetId=dataset_id
    )
    return response['DataSet']

def default_serializer(obj):
    if isinstance(obj, datetime):
        return obj.isoformat()
    raise TypeError(f"Type {type(obj)} not serializable")


ds = get_dataset_description_from_api(account_id, new_dataset_id, region)


with open(f"{new_dataset_id}.json", "w") as f:
    json.dump(ds, f, indent=4, default=default_serializer)

## Compare the updated dataset with the back up, and list the difference(s)

In [68]:
# compare_quicksight_datasets.py

import boto3
import json
import os

def get_dataset_description_from_api(account_id, dataset_id, region='us-east-1'):
    client = boto3.client('quicksight', region_name=region)
    response = client.describe_data_set(
        AwsAccountId=account_id,
        DataSetId=dataset_id
    )
    return response['DataSet']

def get_dataset_description_from_file(filepath):
    with open(filepath, 'r') as f:
        return json.load(f)

def compare_output_columns(old_dataset, new_dataset):
    old_columns = {col["Name"]: col["Type"] for col in old_dataset.get("OutputColumns", [])}
    new_columns = {col["Name"]: col["Type"] for col in new_dataset.get("OutputColumns", [])}

    changes = {
        "datatype_changes": [],
        "added_columns": [],
        "removed_columns": []
    }

    for name, old_type in old_columns.items():
        new_type = new_columns.get(name)
        if new_type is None:
            changes["removed_columns"].append(name)
        elif new_type != old_type:
            changes["datatype_changes"].append({
                "ColumnName": name,
                "OldType": old_type,
                "NewType": new_type
            })

    for name in new_columns:
        if name not in old_columns:
            changes["added_columns"].append({
                "ColumnName": name,
                "NewType": new_columns[name]
            })

    return changes

if __name__ == "__main__":
    # Load environment variables
    account_id = '499080683179'
    new_dataset_id = '55b2a086-8fe0-4d98-921d-509aa4e3bd0d'
    region = 'us-east-1'
    old_json_path = f"{new_dataset_id}.json"  # Default to a local file

    if not all([account_id, new_dataset_id]):
        raise ValueError("Missing required environment variables (QUICKSIGHT_ACCOUNT_ID, NEW_DATASET_ID)")

    # Load old dataset from file
    old_ds = get_dataset_description_from_file(old_json_path)

    # Load new dataset from QuickSight API
    new_ds = get_dataset_description_from_api(account_id, new_dataset_id, region)

    # Compare
    result = compare_output_columns(old_ds, new_ds)

    # Print result
    print("\n--- Comparison Results ---")
    if result["datatype_changes"]:
        print("🛠️ Data Type Changes:")
        for c in result["datatype_changes"]:
            print(f"- {c['ColumnName']}: {c['OldType']} → {c['NewType']}")
    if result["added_columns"]:
        print("\n➕ Added Columns:")
        for c in result["added_columns"]:
            print(f"- {c['ColumnName']}: {c['NewType']}")
    if result["removed_columns"]:
        print("\n➖ Removed Columns:")
        for name in result["removed_columns"]:
            print(f"- {name}")
    if not any(result.values()):
        print("No differences detected.")

    # Save result as JSON
    with open('quicksight_column_diff.json', 'w') as f:
        json.dump(result, f, indent=4)



--- Comparison Results ---
🛠️ Data Type Changes:
- created_at: DATETIME → STRING

➕ Added Columns:
- added: DATETIME


## Back up the analysis definition

In [69]:
import boto3
import json
import os
from datetime import datetime
def get_analysis_definition_from_api(account_id, analysis_id, region):
    client = boto3.client('quicksight', region_name=region)
    response = client.describe_analysis_definition(
        AwsAccountId=account_id,
        AnalysisId=analysis_id
    )
    return response

def default_serializer(obj):
    if isinstance(obj, datetime):
        return obj.isoformat()
    raise TypeError(f"Type {type(obj)} not serializable")

account_id = '499080683179'
analysis_id = '396aa2cb-882a-4aac-ac07-a1c401416e18'
region = 'us-east-1'

analysis = get_analysis_definition_from_api(account_id, analysis_id, 'us-east-1')



with open(f"{analysis_id}.json", "w") as f:
    json.dump(analysis, f, indent=4, default=default_serializer)

## Define the function to remove the broken filter(s)

In [70]:
import boto3

def remove_filters_by_column(analysis_def, dataset_name, column_name):
    """
    Removes timeRangeFilter, and relativeDatesFilter that match a given dataset ID and column name.
    """

    if "FilterGroups" not in analysis_def:
        return analysis_def
        print('No filter is defined in this analysis')

    updated_filter_groups = []

    for fg in analysis_def["FilterGroups"]:
        new_filters = []
        
        for f in fg.get("Filters", []):
            removed = False
            
            print('The filter definition is shown as below')
            print(f)

            for filter_type in ["TimeRangeFilter", "RelativeDatesFilter"]:
                print(filter_type)
                filter_data = f.get(filter_type)
                print(filter_data)
                if filter_data:
                    col = filter_data.get("Column", {})
                    print('The filter is based on this column with changed data type')
                    print(col)
                    if col.get("DataSetIdentifier") == dataset_name and col.get("ColumnName") == column_name:
                        removed = True
                        break  # skip this filter

            if not removed:
                new_filters.append(f)

        if new_filters:
            fg["Filters"] = new_filters
            updated_filter_groups.append(fg)

    analysis_def["FilterGroups"] = updated_filter_groups
    return analysis_def


## Define the function to update the analysis definition by removing the filter

In [71]:
def update_quicksight_analysis(account_id, analysis_id, analysis_def, region):
    """
    Updates an existing QuickSight analysis with the modified definition.
    """
    client = boto3.client("quicksight", region_name=region)

    response = client.update_analysis(
        AwsAccountId=account_id,
        AnalysisId=analysis_id,
        Name=analysis_def["name"],
        Definition=analysis_def["definition"],
        ThemeArn=analysis_def.get("themeArn")
    )

    return response


## Main function to remove the filter and update the analysis

In [72]:
import json
import os

account_id = ''
analysis_id = ''
region = 'us-east-1'
dataset_id = new_dataset_id
column_name = ""
dataset_name = ds["Name"]

# Step 1: Get current analysis definition
client = boto3.client("quicksight", region_name=region)
response = client.describe_analysis_definition(
    AwsAccountId=account_id,
    AnalysisId=analysis_id
)
analysis_def = response["Definition"]


# Step 2: Remove unwanted filters
cleaned_def = remove_filters_by_column(analysis_def, dataset_name, column_name)



# Step 3: Prepare wrapper for update_analysis call
analysis_wrapper = {
    "name": response["Name"],
    "definition": cleaned_def,
    "themeArn": response["ThemeArn"]
}


# Step 4: Update analysis
update_response = update_quicksight_analysis(account_id, analysis_id, analysis_wrapper, region)
print("Update response:", update_response["Status"])


The filter definition is shown as below
{'RelativeDatesFilter': {'FilterId': 'cee584bd-602c-4b23-a58b-1b788dccb735', 'Column': {'DataSetIdentifier': 'hor-comments', 'ColumnName': 'created_at'}, 'AnchorDateConfiguration': {'AnchorOption': 'NOW'}, 'MinimumGranularity': 'DAY', 'TimeGranularity': 'YEAR', 'RelativeDateType': 'LAST', 'RelativeDateValue': 10, 'NullOption': 'NON_NULLS_ONLY'}}
TimeRangeFilter
None
RelativeDatesFilter
{'FilterId': 'cee584bd-602c-4b23-a58b-1b788dccb735', 'Column': {'DataSetIdentifier': 'hor-comments', 'ColumnName': 'created_at'}, 'AnchorDateConfiguration': {'AnchorOption': 'NOW'}, 'MinimumGranularity': 'DAY', 'TimeGranularity': 'YEAR', 'RelativeDateType': 'LAST', 'RelativeDateValue': 10, 'NullOption': 'NON_NULLS_ONLY'}
The filter is based on this column with changed data type
{'DataSetIdentifier': 'hor-comments', 'ColumnName': 'created_at'}
Update response: 202
